In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Cargar archivo
df = pd.read_csv("../../data/raw/JuanSantamaria-cr.csv")

# Renombrar columnas
df.rename(columns={
    "date": "Date", "tmax": "Tmax", "tmin": "Tmin", "tavg": "Tavg",
    "prcp": "Prcp", "wspd": "Wspd", "wdir": "Wdir"
}, inplace=True)

# Convertir columnas numéricas y limitar decimales
for col in ["Tmax", "Tmin", "Tavg", "Prcp", "Wspd"]:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col].fillna(df[col].mean(),)
    df[col] = df[col].round(1)

# Convertir fecha
df["Date"] = pd.to_datetime(df["Date"]).dt.date

# Convertir Wspd a km/h
df["Wspd"] = (df["Wspd"] * 3.6).round(1)

# Asignar dirección del viento (Este en verano, Norte el resto del año)
def direccion_viento_cr(fecha):
    md = (fecha.month, fecha.day)
    if (6, 21) <= md <= (7, 23):
        return "Este"
    else:
        return "Norte"

df["Wdir"] = df["Date"].apply(direccion_viento_cr)

# Agregar columnas fijas
df["Estacion"] = "Juan Santamaria"
df["Provincia"] = "Alajuela"
df["Pais"] = "Costa Rica"

# Eliminar columnas irrelevantes si existen
df.drop(columns=["snow", "wpgt", "pres", "tsun"], inplace=True, errors='ignore')

# Reordenar columnas
df = df[["Date", "Tmax", "Tmin", "Tavg", "Prcp", "Wspd", "Wdir", "Estacion", "Provincia", "Pais"]]

# Guardar
salida = "../../data/cleaned/datos_limpios_juan_santamaria_2024.csv"
os.makedirs(os.path.dirname(salida), exist_ok=True)
df.to_csv(salida, index=False, encoding="utf-8")

print(f"Archivo limpio guardado en: {salida}")